# Unigram tokenization

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

采用类似决策树信息熵的定义方式，一开始把所有子串加入vocabulary，然后每次选择使总loss减少得最少的subwork，剔除vocabulary。总loss定义为每个词的最大出现概率，单个词的最大出现概率（最大拆分）可由DP计算，为每个subword的$-log(P)$相加

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [2]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

In [5]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    print(words_with_offsets)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

[('▁This', (0, 4)), ('▁is', (5, 7)), ('▁the', (8, 11)), ('▁Hugging', (12, 19)), ('▁Face', (20, 24)), ('▁Course.', (25, 32))]
[('▁This', (0, 4)), ('▁chapter', (5, 12)), ('▁is', (13, 15)), ('▁about', (16, 21)), ('▁tokenization.', (22, 35))]
[('▁This', (0, 4)), ('▁section', (5, 12)), ('▁shows', (13, 18)), ('▁several', (19, 26)), ('▁tokenizer', (27, 36)), ('▁algorithms.', (37, 48))]
[('▁Hopefully,', (0, 10)), ('▁you', (11, 14)), ('▁will', (15, 19)), ('▁be', (20, 22)), ('▁able', (23, 27)), ('▁to', (28, 30)), ('▁understand', (31, 41)), ('▁how', (42, 45)), ('▁they', (46, 50)), ('▁are', (51, 54)), ('▁trained', (55, 62)), ('▁and', (63, 66)), ('▁generate', (67, 75)), ('▁tokens.', (76, 83))]


defaultdict(int,
            {'▁This': 3,
             '▁is': 2,
             '▁the': 1,
             '▁Hugging': 1,
             '▁Face': 1,
             '▁Course.': 1,
             '▁chapter': 1,
             '▁about': 1,
             '▁tokenization.': 1,
             '▁section': 1,
             '▁shows': 1,
             '▁several': 1,
             '▁tokenizer': 1,
             '▁algorithms.': 1,
             '▁Hopefully,': 1,
             '▁you': 1,
             '▁will': 1,
             '▁be': 1,
             '▁able': 1,
             '▁to': 1,
             '▁understand': 1,
             '▁how': 1,
             '▁they': 1,
             '▁are': 1,
             '▁trained': 1,
             '▁and': 1,
             '▁generate': 1,
             '▁tokens.': 1})

In [9]:
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

# Sort subwords by frequency
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
print(sorted_subwords[:10])
print(char_freqs)

[('▁t', 7), ('is', 5), ('er', 5), ('▁a', 5), ('▁to', 4), ('to', 4), ('en', 4), ('▁T', 3), ('▁Th', 3), ('▁Thi', 3)]
defaultdict(<class 'int'>, {'▁': 31, 'T': 3, 'h': 9, 'i': 13, 's': 13, 't': 14, 'e': 21, 'H': 2, 'u': 6, 'g': 5, 'n': 11, 'F': 1, 'a': 12, 'c': 3, 'C': 1, 'o': 13, 'r': 9, '.': 4, 'p': 2, 'b': 3, 'k': 3, 'z': 2, 'w': 3, 'v': 1, 'l': 7, 'm': 1, 'f': 1, 'y': 3, ',': 1, 'd': 4})


In [8]:
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
print(token_freqs)
token_freqs = {token: freq for token, freq in token_freqs} #元组拆成字典
print(token_freqs)

[('▁', 31), ('T', 3), ('h', 9), ('i', 13), ('s', 13), ('t', 14), ('e', 21), ('H', 2), ('u', 6), ('g', 5), ('n', 11), ('F', 1), ('a', 12), ('c', 3), ('C', 1), ('o', 13), ('r', 9), ('.', 4), ('p', 2), ('b', 3), ('k', 3), ('z', 2), ('w', 3), ('v', 1), ('l', 7), ('m', 1), ('f', 1), ('y', 3), (',', 1), ('d', 4), ('▁t', 7), ('is', 5), ('er', 5), ('▁a', 5), ('▁to', 4), ('to', 4), ('en', 4), ('▁T', 3), ('▁Th', 3), ('▁Thi', 3), ('▁This', 3), ('Th', 3), ('Thi', 3), ('This', 3), ('hi', 3), ('his', 3), ('th', 3), ('ou', 3), ('se', 3), ('▁tok', 3), ('▁toke', 3), ('▁token', 3), ('tok', 3), ('toke', 3), ('token', 3), ('ok', 3), ('oke', 3), ('oken', 3), ('ke', 3), ('ken', 3), ('▁s', 3), ('ra', 3), ('nd', 3), ('▁i', 2), ('▁is', 2), ('▁th', 2), ('▁the', 2), ('the', 2), ('he', 2), ('▁H', 2), ('in', 2), ('rs', 2), ('te', 2), ('▁ab', 2), ('ab', 2), ('▁tokeni', 2), ('▁tokeniz', 2), ('tokeni', 2), ('tokeniz', 2), ('okeni', 2), ('okeniz', 2), ('keni', 2), ('keniz', 2), ('eni', 2), ('eniz', 2), ('ni', 2), ('ni

In [11]:
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
print(total_sum)
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}
print(model)

594
{'▁': 2.952892114877499, 'T': 5.288267030694535, 'h': 4.189654742026425, 'i': 3.821929961901108, 's': 3.821929961901108, 't': 3.7478219897473863, 'e': 3.342356881639222, 'H': 5.6937321388027, 'u': 4.59511985013459, 'g': 4.777441406928545, 'n': 3.9889840465642745, 'F': 6.386879319362645, 'a': 3.9019726695746444, 'c': 5.288267030694535, 'C': 6.386879319362645, 'o': 3.821929961901108, 'r': 4.189654742026425, '.': 5.000584958242754, 'p': 5.6937321388027, 'b': 5.288267030694535, 'k': 5.288267030694535, 'z': 5.6937321388027, 'w': 5.288267030694535, 'v': 6.386879319362645, 'l': 4.440969170307332, 'm': 6.386879319362645, 'f': 6.386879319362645, 'y': 5.288267030694535, ',': 6.386879319362645, 'd': 5.000584958242754, '▁t': 4.440969170307332, 'is': 4.777441406928545, 'er': 4.777441406928545, '▁a': 4.777441406928545, '▁to': 5.000584958242754, 'to': 5.000584958242754, 'en': 5.000584958242754, '▁T': 5.288267030694535, '▁Th': 5.288267030694535, '▁Thi': 5.288267030694535, '▁This': 5.28826703069453

In [13]:
def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)): #区间DP算法，找到使对数概率和最大的分割
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

In [14]:
print(encode_word("Hopefully", model))
print(encode_word("This", model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 41.5157494601402)
(['This'], 6.288267030694535)


In [15]:
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

In [16]:
compute_loss(model)

413.10377642940875

In [17]:
import copy


def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

In [18]:
scores = compute_scores(model)
print(scores["ll"])
print(scores["his"])

6.376412403623874
0.0


In [19]:
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])

    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

In [20]:
def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])


tokenize("This is the Hugging Face course.", model)

['▁This',
 '▁is',
 '▁the',
 '▁Hugging',
 '▁Face',
 '▁',
 'c',
 'ou',
 'r',
 's',
 'e',
 '.']